<a href="https://colab.research.google.com/github/claudiosegala/Monografia/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [4]:
!pip install tensorflow
!pip install panda

  Stored in directory: /root/.cache/pip/wheels/c6/c8/45/06ed898b0bb401c1ff207dbb05b1587ff28860a236d98b1996
Successfully built panda


# Define headers

In [0]:
import tensorflow as tf # machine learning library
import panda as pd # data manipulation library

In [0]:
data = pd.read_csv("https://storage.googleapis.com/ml_universities/california_housing_train.csv", sep=",")
